In [1]:
!pip install ensemble_boxes

     |████████████████████████████████| 3.4 MB 2.4 MB/s eta 0:00:01     |███████████████████████▍        | 2.5 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 12.1 MB/s eta 0:00:01    |███▋                            | 2.8 MB 10.6 MB/s eta 0:00:03     |████▍                           | 3.4 MB 10.6 MB/s eta 0:00:03     |█████████                       | 7.0 MB 10.6 MB/s eta 0:00:02     |██████████████████▋             | 14.7 MB 12.1 MB/s eta 0:00:01     |███████████████████████████▊    | 21.9 MB 12.1 MB/s eta 0:00:01


In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [17]:
# ensemble csv files
submission_files = ['best.csv', '0.5312.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [18]:
image_ids = submission_df[0]['image_id'].tolist()

In [19]:
annotation = '../../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [20]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [21]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble_5312_5508.csv')

submission.iloc[0]["PredictionString"]

'1 0.87252027 164.55457 75.88126 255.48257 165.87816 1 0.55075884 184.18134 363.0596 205.64175 396.8725 4 0.062581174 184.09273 362.6412 205.71352 396.87146 8 0.9996784 171.76382 234.75102 461.35687 471.81796 8 0.20859158 183.85115 363.06308 211.14252 397.7394 '

In [22]:
submission

,PredictionString,image_id
0,1 0.87252027 164.55457 75.88126 255.48257 165....,batch_01_vt/0021.jpg
1,1 0.9565772 0.93178713 246.9561 307.15573 458....,batch_01_vt/0028.jpg
2,1 0.9996809 71.261955 167.88586 373.51526 378....,batch_01_vt/0031.jpg
3,2 0.9972133 298.36743 159.87602 363.3463 226.7...,batch_01_vt/0032.jpg
4,0 0.122315705 111.71272 252.02997 281.51892 28...,batch_01_vt/0070.jpg
...,...,...
832,1 0.9089043 73.766754 157.16164 412.03818 383....,batch_03/0947.jpg
833,1 0.6266063 410.60712 370.1818 425.43204 412.7...,batch_03/0968.jpg
834,0 0.054004665 69.905334 93.51987 80.1509 114.9...,batch_03/0969.jpg
835,0 0.07236068 147.17018 175.29332 200.14139 254...,batch_03/0992.jpg
